In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from train import LitTransformer
import torch
import yaml
from data import get_tgt_str, build_vocab, id2token

In [9]:
config_path = "/home/takeru/AlphaSymbol/models/config_20241206174837.yaml"
state_dict = "/home/takeru/AlphaSymbol/models/model_30000epochs_20241206182634.pth"

with open(config_path, "r") as f:
    config = yaml.safe_load(f)

src_max_len = config["src_max_len"]
tgt_max_len = config["tgt_max_len"]
src_vocab = config["src_vocab"]
tgt_vocab = config["tgt_vocab"]

model = LitTransformer(len(src_vocab), len(tgt_vocab), src_max_len, tgt_max_len,
        d_model=64,
        nhead=4,
        num_encoder_layers=6,
        num_decoder_layers=6,
        dim_feedforward=512,
        dropout=0.1,)

inputs = [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,)]
#outputs = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
#outputs = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
outputs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
src_str = get_tgt_str(inputs, outputs)
src_tensor = torch.tensor([src_vocab[i] for i in src_str])
src_tensor = torch.cat([src_tensor, torch.tensor([src_vocab["<pad>"] for _ in range(src_max_len - src_tensor.shape[0])])])
src_tensor = src_tensor.unsqueeze(0)
out_str = torch.tensor(tgt_vocab["<sos>"]).reshape((1, ))
out_str = out_str.unsqueeze(0)

model.load_state_dict(torch.load(state_dict, weights_only=True))
model.eval()

with torch.no_grad():
    for i in range(tgt_max_len):
        pred = model(src_tensor, out_str)
        next_c = pred[0, -1, :]
        max_values, max_indices = torch.max(next_c, axis=0)
        out_str = torch.cat((out_str, max_indices.reshape(1, 1)), axis=1)
        if max_indices == tgt_vocab["<eos>"]:
            break

print([id2token(tgt_vocab, id) for id in out_str[0]])

['<sos>', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S']
